In [1]:
import cobra
import GEMS
import pandas as pd
import numpy as np
import os
import re

import pickle

from etcpy import etc
from etcpy import tempDep

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

In [2]:
data = GEMS.load_exp_data('../data/ExpGrowth.csv')
path = '../'
params = pd.read_csv(os.path.join(path,'data/model_enzyme_params_new_tagged.csv'),index_col=0)
df = etc.calculate_thermal_params(params)

dfae_batch = data[0].set_index('Ts').rename_axis(None)
params

,Topt,Topt_std,Length,Tm,Tm_std,T90,dCpt,dCpt_std,topt_source,TmTag
P77467,306.15,10,262,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76077,311.15,10,309,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76078,310.15,10,95,328.717325,7.588735,NaN,-4000,1000,BullShit,Mean
P76079,309.15,10,248,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76081,310.15,10,356,328.717325,7.588735,NaN,-4000,1000,BullShit,Mean
...,...,...,...,...,...,...,...,...,...,...
P0AG80,313.15,10,438,329.166303,4.070000,330.45,-4000,1000,predicted,Exp
P77718,307.15,10,482,319.125751,4.070000,320.15,-4000,1000,predicted,Exp
P0A6D5,308.15,10,288,328.762604,7.710000,NaN,-4000,1000,predicted,Mean
standard,308.15,10,272,328.762604,7.710000,NaN,-4000,1000,BullShit,Mean


In [3]:
def aerobic(params):
    # thermalParams: a dictionary with ids like uniprotid_Topt 
    df = etc.calculate_thermal_params(params)
    #mae = pickle.load(open(os.path.join(path,'models/aerobic.pkl'),'rb'))
    mae = cobra.io.load_matlab_model('../models/ecModel.mat')
    
    match = re.compile('usage_prot_*', re.IGNORECASE)
    # for react in mae.reactions:
    #     if match.match(react.id) and react.lower_bound != -1000:
    #         ori_lb = react.lower_bound
    #         react.lower_bound = ori_lb*2.5


    mae.reactions.EX_glc__D_e.lower_bound = -100
    mae.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound = 50
    #mae.medium['EX_glc__D_e'] = 100
    
    try: 
        rae, gae, oae, cae, aae = etc.simulate_growth(mae,dfae_batch.index+273.15,df=df,sigma=0.5)
    except: 
        rae = np.zeros(dfae_batch.shape[0])
        model = None
    
    rae = [0 if x is None else x for x in rae]
    rae = [0 if x<1e-3 else x for x in rae]
    rexp = dfae_batch['r'].values
    #print(rae)
    print(abs(rae - dfae_batch['r'].values))
    print('r2_batch:',r2_score(rexp,rae))
    print('MSE_ae',MSE(rexp,rae))
    return {'growth':np.array(rae), 'glucose':np.array(gae), 'oxygen': np.array(oae), 'carbonDioxide': np.array(cae), 'acetate': np.array(aae)}, r2_score(rexp,rae), MSE(rexp,rae)

In [4]:
df = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])

for i in range(1,11):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_random_topt(params, ['Topt'])
    rae, score, mse = aerobic(newParams)
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-06
Growth at  20.153299075508016 is:  0.2815466836805465
Growth at  23.00151111308503 is:  0.37080204870620953
Growth at  26.331896319367047 is:  0.4790469735884477
Growth at  28.251819864188008 is:  0.5337388339191941
Growth at  29.99421176442104 is:  0.5782831684251486
Growth at  31.62212188446705 is:  0.6000140952221431
Growth at  33.61728633658504 is:  0.5611739784108365
Growth at  36.55299482796005 is:  0.49259496745538495
Growth at  38.54155004207803 is:  0.44170758858047343
Growth at  40.20094365636703 is:  0.3983349371521727
Growth at  42.23876395989504 is:  0.3456602283667271
Growth at  44.453491097574044 is:  0.28530025997026076
Growth at  45.28989924561603 is:  0.26252529286219034
Growth at  46.64533098816804 is:  0.22803575892453576
Growth at  48.47717620988101 is:  0.1863292743412915
[0.01118668 0.21113795 0.15775703 0.24944117 0.37780483 0.4923339
 0.62445002 0.82356503 0.99732041 0.

In [5]:
df.to_csv('../results/onlyKcat.csv')

In [6]:
df

,iter,temp,growth,o2flux,gluFlux,co2flux,acetateFlux,R2,MSE
0,1,20.153299,0.281547,8.685062,6.778438,-7.514281,-0.164022,-1.925769,0.450280
1,1,23.001511,0.370802,10.814416,7.767799,-9.870026,-0.335951,-1.925769,0.450280
2,1,26.331896,0.479047,12.897703,9.266835,-12.152261,-1.285686,-1.925769,0.450280
3,1,28.251820,0.533739,14.464211,10.797613,-13.389017,-1.458320,-1.925769,0.450280
4,1,29.994212,0.578283,15.296391,9.783552,-15.302056,-2.459392,-1.925769,0.450280
...,...,...,...,...,...,...,...,...,...
10,10,42.238764,0.349821,9.479238,4.236940,-10.225684,-0.417561,-1.909237,0.447736
11,10,44.453491,0.289829,8.057167,3.529955,-8.675603,-0.296996,-1.909237,0.447736
12,10,45.289899,0.266870,7.509779,3.259946,-8.079224,-0.256905,-1.909237,0.447736
13,10,46.645331,0.232058,6.674737,2.853859,-7.169902,-0.208563,-1.909237,0.447736
